<a href="https://colab.research.google.com/github/karthebans/ANN-Bank_Customer_Churn/blob/main/FInetuning_of_Mistral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install accelerate peft bitsandbytes trl py7zr auto-gptq optimum transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 8.2 MB/s eta 0:00:00


In [11]:
# from huggingface_hub import login

# login(token="your_huggingface_token")

In [12]:
from huggingface_hub import notebook_login
notebook_login()

In [13]:
import torch
import os
from datasets import load_dataset,Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments,GPTQConfig
from peft import LoraConfig, AutoPeftModelForCausalLM, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer

In [14]:
data=load_dataset("samsum",split="train")

In [15]:
data

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
})

In [16]:
data.to_pandas()

,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,13681000,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,13728094,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."
...,...,...,...
14727,13863028,Romeo: You are on my ‘People you may know’ lis...,Romeo is trying to get Greta to add him to her...
14728,13828570,Theresa: <file_photo>\r\nTheresa: <file_photo>...,Theresa is at work. She gets free food and fre...
14729,13819050,John: Every day some bad news. Japan will hunt...,Japan is going to hunt whales again. Island an...
14730,13828395,Jennifer: Dear Celia! How are you doing?\r\nJe...,Celia couldn't make it to the afternoon with t...


In [17]:
data.to_pandas().iloc[:,1][0]

"Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)"

In [18]:
data.to_pandas().iloc[:,2][0]

'Amanda baked cookies and will bring Jerry some tomorrow.'

In [19]:
data_df=data.to_pandas()

LLM Input:

###Human: Summarize this following dialogue:  

Amanda: I baked cookies. Do you want some?  
Jerry: Yes, I’d love some.  

### AI Assistant(Model):

Amanda baked cookies and will bring Jerry some.


In [20]:
data_df.head()

,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,13681000,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,13728094,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."
